# Generate events

This guide shows how to use the Madgraph5 class of HML to generate events from Z
boson and QCD to dijets processes. To get started, let’s import some necessary
classes from generators module:

In [1]:
from hml.generators import Madgraph5, Madgraph5MultiRun, Madgraph5Run

Welcome to JupyROOT 6.24/02


## Initialization

The Madgraph5 API (application programming interface) provided by HML works
similarly to the Madgraph5 CLI (command line interface): initialization
corresponds to the classic workflow of `generate`, `output`, `launch`.

In [2]:
zjj = Madgraph5(
    executable="mg5_aMC",
    model="sm",
    definitions={},
    processes=["p p > z z, z > j j, z > vl vl~"],
    output="data/pp2zz_z2jj_z2vlvl",
)

- `executable` refers to the path of the `mg5_aMC` executable file.
- `model` is managed by Madgraph5 itself.
- `definitions` are used for the `define` command in Madgraph5.
- Usually in Madgraph5, you need to use `generate` and `add process` to create
multiple processes. In HML, processes are represented as a list of strings.
- If `output` already exists, this class will not generate processes again.

## Launch the first run

After initializing the class (or `output` in Madgraph5 CLI), you can now launch
a new run:

In [3]:
zjj.launch(
    shower="pythia8",
    detector="delphes",
    settings={"iseed": 42, "nevents": 1000, "htjmin": 400},
)

Running Survey
Running Pythia8
Running Delphes
Storing files

Done


Madgraph5MultiRun(name='run_01', tag='tag_1', seed=42, n_events=1000, cross_section=0.0022735, collider='pp: 6500.0x6500.0', events=<cppyy.gbl.TChain object at 0x818cf10>, runs=[Madgraph5Run(name='run_01_0', tag='tag_1', directory=PosixPath('/root/workspace_ssd/projects/hep-ml-lab/examples/data/pp2zz_z2jj_z2vlvl/Events/run_01_0'), seed=42, n_events=1000, cross_section=0.0022735, collider='pp: 6500.0x6500.0', events=<cppyy.gbl.TChain object at 0x7760280>)])

- `shower` and `detector` are options for parton shower and detector simulation
tools. Currently, `pythia8` and `delphes` are available.
- In Madgraph5, you can use the `set` command to change configurations in
different cards without opening them. The `settings` attribute contains these
configurations as a Python dictionary.
- To generate a large number of events, set the `multi_run` parameter.

## Check the information

After the generation is finished, you can use `.summary()` to check all the
information of runs:

In [4]:
zjj.summary()

                 p p > z z, z > j j, z > vl vl~                 
┏━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┓
┃ # ┃ Name      ┃ Tag   ┃ Cross section (pb) ┃ N events ┃ Seed ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━┩
│ 0 │ run_01[1] │ tag_1 │     2.273e-03      │    1,000 │   42 │
└───┴───────────┴───────┴────────────────────┴──────────┴──────┘
                            Output:                             
/root/workspace_ssd/projects/hep-ml-lab/examples/data/pp2zz_z2jj
                            _z2vlvl                             

The data in the summary table are properties of a run:

In [5]:
run_01 = zjj.runs[0]
print("Name:", run_01.name)
print("N Subruns:", len(run_01.runs))
print("Tag:", run_01.tag)
print("Cross section:", run_01.cross_section)
print("N events:", run_01.n_events)
print("Seed:", run_01.seed)

Name: run_01
N Subruns: 1
Tag: tag_1
Cross section: 0.0022735
N events: 1000
Seed: 42


You can access the `events` attribute to read the root file from Delphes:

In [6]:
for event in run_01.events:
    print(f"n_jets: {event.Jet_size}")
    print(f"n_fat_jets: {event.FatJet_size}")
    break

n_jets: 2
n_fat_jets: 1


## Launch the second run similarly

To build a signal vs background binary classification task, we need to generate
QCD events without the intermediate Z boson.

In [7]:
qcd = Madgraph5(
    processes=['p p > j j / z'],
    output="./data/qcd"
)

qcd.launch(
    shower="pythia8",
    detector="delphes",
    settings={"iseed": 42, "nevents": 1000, "htjmin": 400},
)

qcd.summary()

Running Survey
Running Pythia8
Running Delphes
Storing files

Done


                         p p > j j / z                          
┏━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━┓
┃ # ┃ Name      ┃ Tag   ┃ Cross section (pb) ┃ N events ┃ Seed ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━┩
│ 0 │ run_01[1] │ tag_1 │     5.611e+04      │    1,000 │   42 │
└───┴───────────┴───────┴────────────────────┴──────────┴──────┘
                            Output:                             
   /root/workspace_ssd/projects/hep-ml-lab/examples/data/qcd    

## Read the existing output

HML can handle three cases:

1. Events are generated by HML:

In [8]:
Madgraph5.from_output("./data/pp2zz_z2jj_z2vlvl")

2. Events are generated by the `multi_run` command from MadEvent:

In [9]:
Madgraph5MultiRun.from_name("run_01", "./data/pp2zz_z2jj_z2vlvl")

Madgraph5MultiRun(name='run_01', tag='tag_1', seed=42, n_events=1000, cross_section=0.0022735, collider='pp: 6500.0x6500.0', events=<cppyy.gbl.TChain object at 0x1e4bbb0>, runs=[Madgraph5Run(name='run_01_0', tag='tag_1', directory=PosixPath('data/pp2zz_z2jj_z2vlvl/Events/run_01_0'), seed=42, n_events=1000, cross_section=0.0022735, collider='pp: 6500.0x6500.0', events=<cppyy.gbl.TChain object at 0x83f6f80>)])

3. Events are generated normally by the `launch` command from Madgraph5:

In [10]:
Madgraph5Run.from_directory("./data/pp2zz_z2jj_z2vlvl/Events/run_01_0")

Madgraph5Run(name='run_01_0', tag='tag_1', directory=PosixPath('data/pp2zz_z2jj_z2vlvl/Events/run_01_0'), seed=42, n_events=1000, cross_section=0.0022735, collider='pp: 6500.0x6500.0', events=<cppyy.gbl.TChain object at 0x1e3fe40>)

For more information about `Madgraph5, Madgraph5MultiRun, Madgraph5Run`, refer
to this (link to api).